# SP API Examples
This notebook is an bare-bone example on how to query Amazon SP API.
Useful for API exploration and debugging purposes.

In [1]:
import boto3
import requests
import json
import json
import io
import csv
from botocore.exceptions import ClientError

### Get aws_session_token from your credentials file

In [ ]:
import configparser

file_path = "/Users/emilianofrigo/.aws/credentials"

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the file
config.read(file_path)

# Access the value of aws_session_token inside the [data-prod] section
aws_session_token = config.get('data-prod', 'aws_session_token')

print("AWS Session Token:", aws_sessidon_token)

## Configuration

In [3]:
AWS_CRED_PROFILE_NAME = 'data-prod'
AWS_SESSION_TOKEN = aws_session_token

# config this
account_name = "boba"
account_region = "NA"
# marketplace_id = "A1F83G8C2ARO7P" # developer-docs.amazon.com/sp-api/docs/marketplace-ids -- UK
marketplace_id = "ATVPDKIKX0DER" # developer-docs.amazon.com/sp-api/docs/marketplace-ids  -- US
base_url = "https://sellingpartnerapi-na.amazon.com/"

account_secret_key = "prod/airbyte/sources/sp_api/"+account_name

## Authentication

In [4]:
def get_secret_by_name(secret_name: str):
    secret_str = _get_secret(secret_name)
    return json.loads(secret_str)


def _get_secret(secret_name: str, region_name: str = "eu-west-1"):
    session = boto3.session.Session(profile_name=AWS_CRED_PROFILE_NAME)
    client = session.client(service_name="secretsmanager", region_name=region_name)

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise e

    return get_secret_value_response["SecretString"]

In [ ]:
secret = get_secret_by_name(account_secret_key)
secret

In [6]:
# Get access token
url = "https://api.amazon.com/auth/o2/token"
client_secret=secret["lwa_client_secret"]
refresh_token=secret["refresh_tokens"][account_region]
client_id = secret["lwa_app_id"]

payload=f'grant_type=refresh_token&refresh_token={refresh_token}&client_id={client_id}&client_secret={client_secret}'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)
access_token = response.json()["access_token"]

## GET_SELLER_FEEDBACK_DATA

In [ ]:
# Create report
reportType = "GET_SELLER_FEEDBACK_DATA"
url = f"{base_url}reports/2021-06-30/reports"

payload={
    "marketplaceIds":[marketplace_id],
    "reportType":reportType,
    "reportOptions":{}
}

headers = {
  'Accept': 'application/json',
  'x-amz-access-token': access_token,
  'X-Amz-Security-Token': AWS_SESSION_TOKEN,
  "content-type" : "application/json; charset=utf-8"
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

try:
  report_id = response.json()['reportId']
except KeyError as e:
  print(f"error: {e}")

response.json()

In [54]:
# Or set report id manually (see below for report request history)
# report_id = 3000121019934

In [ ]:
## get the report using the report_id
url = f"{base_url}/reports/2021-06-30/reports/{report_id}"

headers = {
  'Accept': 'application/json',
  'x-amz-access-token': access_token,
  'X-Amz-Security-Token': AWS_SESSION_TOKEN,
  "content-type" : "application/json; charset=utf-8"
}

response = requests.request("get", url, headers=headers)
report = response.json()
report

In [ ]:
response.headers

## Get requested reports history (simple request)

In [ ]:
url = f"{base_url}/reports/2021-06-30/reports"

headers = {
  'Accept': 'application/json',
  'x-amz-access-token': access_token,
  'X-Amz-Security-Token': AWS_SESSION_TOKEN,
  "content-type" : "application/json; charset=utf-8"
}

payload = {
    'reportTypes': ['GET_SELLER_FEEDBACK_DATA'],
}

response = requests.request("get", url, headers=headers, params=payload)
response.json()


## Get requested reports history (full tokenized response)

In [ ]:
def fetch_reports(url, headers, params):
    all_reports = []
    next_token = None

    while True:
        request_params = params.copy()
        if next_token:
            request_params = {'nextToken': next_token}  # Only include nextToken if present
        
        response = requests.get(url, headers=headers, params=request_params)
        
        data = response.json()
        
        reports = data.get('reports', [])
        
        all_reports.extend(reports)
        
        # Check for the nextToken
        next_token = data.get('nextToken')
        
        if not next_token:
            break  # No more pages, exit the loop

    return all_reports


# Initial parameters for the request
url = f"{base_url}/reports/2021-06-30/reports"

headers = {
    'Accept': 'application/json',
    'x-amz-access-token': access_token,
    'X-Amz-Security-Token': AWS_SESSION_TOKEN,
    'Content-Type': 'application/json; charset=utf-8'
}

initial_params = {
    'reportTypes': 'GET_SELLER_FEEDBACK_DATA'
}

# Fetch all reports
all_reports = fetch_reports(url, headers, initial_params)

print(f"Total number of elements in response: {len(all_reports)}")

# Uncomment to check the whole response
# print(json.dumps(all_reports, indent=2))

#  Filter relevant marketplace
marketplace_reports = [report for report in all_reports if marketplace_id in report['marketplaceIds']]

print(f"Reports for current marketplace: '{marketplace_id}': {len(marketplace_reports)}")
print(json.dumps(marketplace_reports, indent=2))


### Filter last DONE report

In [ ]:
done_reports = [report for report in all_reports if report['processingStatus'] == 'DONE' and marketplace_id in report['marketplaceIds']]

print(f"Filtered DONE elements for marketplace '{marketplace_id}'. Total DONE elements in response: {len(done_reports)}")
print(json.dumps(done_reports[:1], indent=2))

## Get document of the Report

In [16]:
# copy reportDocumentId from DONE report
report_doc_id = "amzn1.spdoc.1.4.na.fbed5ffe-2c7f-4931-9d0c-2308ae76e3a7.T1EQWNPFMZIIYZ.3001"
url = f"{base_url}/reports/2021-06-30/documents/{report_doc_id}"

headers = {
  'Accept': 'application/json',
  'x-amz-access-token': access_token,
  'X-Amz-Security-Token': AWS_SESSION_TOKEN,
  "content-type" : "application/json; charset=utf-8"

}

response = requests.request("get", url, headers=headers)

In [ ]:
# UNCOMMENT FOR DEBUG INFO
# response.headers

In [ ]:
response.json()

In [ ]:
url = response.json()["url"]
url

In [ ]:
report = requests.get(url)
report.content

In [ ]:
f = io.StringIO(report.content.decode())
json_data = [d for d in csv.DictReader(f, delimiter='\t')]
json_data

# Specify the path where you want to save the JSON file
file_path = f"report_samples/{account_name}_{account_region}_{reportType}.json"

# Write the JSON data to the specified file
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"JSON data has been written to {file_path}")